<a href="https://colab.research.google.com/github/Darshu21/dars/blob/main/Semantic_Search_%2B_GPT_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers pinecone-client openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.1 MB/s eta 0:00:00


In [2]:
import pinecone
from sentence_transformers import SentenceTransformer,util
import requests
from bs4 import BeautifulSoup
model = SentenceTransformer('all-MiniLM-L6-v2') #384 dimensional

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
model.encode("Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions").tolist()

[0.008430823683738708,
 -0.047467492520809174,
 -0.0466928631067276,
 -0.052048128098249435,
 0.01978888176381588,
 0.052723295986652374,
 0.04106181487441063,
 0.04269367828965187,
 -0.0046617924235761166,
 0.009379039518535137,
 0.02730366215109825,
 -0.08771481364965439,
 0.015284479595720768,
 0.027853088453412056,
 0.055015720427036285,
 -0.03243939206004143,
 -0.09376099705696106,
 -0.03404098004102707,
 0.04118771106004715,
 -0.05621286481618881,
 0.03625637665390968,
 -0.08846645802259445,
 -0.05318570137023926,
 -0.05722220614552498,
 -0.010659565217792988,
 -0.0016292507061734796,
 0.00048383502871729434,
 0.004385568201541901,
 -0.04214197397232056,
 -0.010113080963492393,
 0.03503518924117088,
 -0.07891512662172318,
 0.0630989670753479,
 -0.028103651478886604,
 -0.03996216505765915,
 0.06134173274040222,
 0.05923076719045639,
 0.04462162405252457,
 0.0026261138264089823,
 0.03360328450798988,
 -0.009221777319908142,
 -0.11060938984155655,
 -0.04329496994614601,
 -0.05028435

In [4]:
def get_html_content(url):
    response = requests.get(url)
    return response.content

In [5]:
def get_plain_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    for script in soup(["script"]):
        script.extract()
    return soup.get_text()

In [6]:
def split_text_into_chunks(plain_text, max_chars=2000):
    text_chunks = []
    current_chunk = ""
    for line in plain_text.split("\n"):
        if len(current_chunk) + len(line) + 1 <= max_chars:
            current_chunk += line + " "
        else:
            text_chunks.append(current_chunk.strip())
            current_chunk = line + " "
    if current_chunk:
        text_chunks.append(current_chunk.strip())
    return text_chunks

In [7]:
def scrape_text_from_url(url, max_chars=2000):
    html_content = get_html_content(url)
    plain_text = get_plain_text(html_content)
    text_chunks = split_text_into_chunks(plain_text, max_chars)
    return text_chunks

In [8]:
url = input("Enter the URL to scrape text from: ")
plain_text_chunks = scrape_text_from_url(url)
print(plain_text_chunks)

Enter the URL to scrape text from: https://en.wikipedia.org/wiki/Virat_Kohli
['Virat Kohli - Wikipedia                                   Jump to content        Main menu      Main menu move to sidebar hide    \t\tNavigation \t   Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us      \t\tContribute \t   HelpLearn to editCommunity portalRecent changesUpload file                    Search            Search                       Appearance                 Donate  Create account  Log in         Personal tools      Donate Create account Log in      \t\tPages for logged out editors learn more    ContributionsTalk                             Contents move to sidebar hide     (Top)      1 Early life         2 Career         3 Player profile     Toggle Player profile subsection      3.1 Comparisons to Sachin Tendulkar         3.2 Playing style         3.3 Aggression           4 Public image and in media         5 Personal life         6 Outside cricket     Toggle Outside cri

In [9]:
len(plain_text_chunks)

46

In [10]:
plain_text_chunks[10]

'ESPN ranked Kohli among the world\'s most renowned athletes.[74][75] In 2014, American Appraisal conducted an evaluation of Kohli\'s brand value and determined it to be $56.4 million, placing him fourth on the list of India\'s most valued celebrity brands.[76] In October 2016 by Duff & Phelps, Kohli\'s brand worth had increased to $92 million, second only to that of Bollywood actor Shah Rukh Khan.[77] In 2017, Kohli was recognized by Forbes as the seventh "Most Valuable Brand among Athletes", with a brand estimation of $14.5 million.[78] By September of the same year, Kohli had endorsement agreements with 17 distinct brands, and he announced that he would only endorse products that he personally uses and believes in.[79] The following year, he was featured on Time magazine\'s annual list of the 100 most influential people in the world.[80] In 2019, Kohli was the only cricketer included in Forbes\' list of "World\'s 100 Highest-Paid Athletes". He was ranked 100th on the list, with earn

In [13]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key="pcsk_CxRFo_GmE3sUvunVMe9oZouMdg2AFHNAng5j7h24Euodj9atvKjC9gLchgKgcDD1wqFBK" )

    # Now do stuff
if 'darshu' not in pc.list_indexes().names():
        pc.create_index(
            name='darshu',
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

In [16]:
index.describe_index_stats()

NameError: name 'index' is not defined

In [ ]:
def addData(corpusData,url):
    id  = index.describe_index_stats()['total_vector_count']
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.encode(chunk).tolist(),
                {'title': url,'context': chunk})
        index.upsert(vectors=[chunkInfo])

In [ ]:
plain_text_chunks[10]

'Raj Singh Dungarpur is credited for the selection of Tendulkar for the Indian tour of Pakistan in late 1989,[67] after one first class season.[68] The Indian selection committee had shown interest in selecting Tendulkar for the tour of the West Indies held earlier that year, but eventually did not select him, as they did not want him to be exposed to the dominant fast bowlers of the West Indies so early in his career. Tendulkar made his Test debut against Pakistan in Karachi in November 1989 aged 16 years and 205 days. He made 15 runs, being bowled by Waqar Younis, who also made his debut in that match, but was noted for how he handled numerous blows to his body at the hands of the Pakistani pace attack.[69] In the fourth and final Test in Sialkot, he was hit on the nose by a bouncer bowled by Younis, but he declined medical assistance and continued to bat even as he gushed blood from it.[70] In a 20-over exhibition game in Peshawar, held in parallel with the bilateral series, Tendulk

In [ ]:
addData(plain_text_chunks,url)

In [ ]:
def find_match(query,k):
    query_em = model.encode(query).tolist()
    result = index.query(query_em, top_k=k, includeMetadata=True)

    return [result['matches'][i]['metadata']['title'] for i in range(k)],[result['matches'][i]['metadata']['context'] for i in range(k)]

In [ ]:
find_match("What awards was tendulakar awarded with?",2)

(['https://en.wikipedia.org/wiki/Sachin_Tendulkar',
  'https://en.wikipedia.org/wiki/Sachin_Tendulkar'],
 ['Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India internationally for close to twenty-four years. In 2002, halfway through his career, Wisden ranked him the second-greatest Test batsman of all time, behind Don Bradman, and the second-greatest ODI batsman of all time, behind Viv Richards.[8] Later in his career, Tendulkar was part of the Indian team that won the 2011 Cricket World Cup, his first win in six World Cup appearances for India.[9] He had previously been named "Player of the Tournament" at the 2003 edition of the tournament. Tendulkar received the Arjuna Award in 1994 for his outstanding sporting achievements, the Khel Ratna Award, India\'s highest sporting honour, in 1997, and the Padma Shri and Padma Vibhushan awards in 1

###Prompt
Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

In [ ]:
import openai
openai.api_key="openai key"
def create_prompt(context,query):
    header = "Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

def generate_answer(prompt):
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response.choices[0].text).strip()

In [ ]:
query = "Who is Sachin?"
docs,res = find_match(query,1)

In [ ]:
res

['^ "Sachin Tendulkar: Who is he?". India Today. 3 April 2012. Retrieved 30 August 2021.  ^ Ghosh, Avijit (5 January 2008). "Now, Oz-speak about Brahmin dominance!". The Times of India. Retrieved 30 August 2021.  ^ Thani, L.; Mishra, R. (1999). Sensational Sachin. Diamond Pocket Books. p.\xa0113. ISBN\xa081-288-2573-9. His mother Rajni Tendulkar worked in L.I.C.  ^ "Ajit Dotes on Sachin, Who Could\'ve Guessed They\'re Half-Brothers!". The Quint. Retrieved 2 September 2018.  ^ "This man sacrificed his cricket, career for Sachin Tendulkar". Asianet Newsable.  ^ "38 special facts about Sachin Tendulkar – 3". MSN. 16 March 2012. Archived from the original on 6 June 2013. Retrieved 18 December 2012.  ^ a b "\'कांबली बहुत गुस्सैल है,\' सचिन ने KBC में अमिताभ को सुनाई थी दोस्त की कहानी; देखें Video". Jansatta (in Hindi). Retrieved 24 December 2021.  ^ "38 special facts about Sachin Tendulkar – 2". MSN. 16 March 2012. Archived from the original on 6 June 2013. Retrieved 18 December 2012.  ^ "T

In [ ]:
context= "\n\n".join(res)
prompt = create_prompt(context,query)
print(prompt)

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' 
^ "Sachin Tendulkar: Who is he?". India Today. 3 April 2012. Retrieved 30 August 2021.  ^ Ghosh, Avijit (5 January 2008). "Now, Oz-speak about Brahmin dominance!". The Times of India. Retrieved 30 August 2021.  ^ Thani, L.; Mishra, R. (1999). Sensational Sachin. Diamond Pocket Books. p. 113. ISBN 81-288-2573-9. His mother Rajni Tendulkar worked in L.I.C.  ^ "Ajit Dotes on Sachin, Who Could've Guessed They're Half-Brothers!". The Quint. Retrieved 2 September 2018.  ^ "This man sacrificed his cricket, career for Sachin Tendulkar". Asianet Newsable.  ^ "38 special facts about Sachin Tendulkar – 3". MSN. 16 March 2012. Archived from the original on 6 June 2013. Retrieved 18 December 2012.  ^ a b "'कांबली बहुत गुस्सैल है,' सचिन ने KBC में अमिताभ को सुनाई थी दोस्त 

In [ ]:
reply = generate_answer(prompt)
print(reply)

Sachin Tendulkar is an Indian cricketer widely regarded as one of the greatest batsmen in the history of cricket. He is the highest run scorer of all time in International cricket. He is the only player to have scored one hundred international centuries, the first batsman to score a double century in a One Day International, and the only player to complete more than 30,000 runs in international cricket.
